# MNLP Homework 2 - OCRed text cleaning

In [ ]:
import google.generativeai as genai
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
import json
import nltk
import sys
import os


from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from nltk.tokenize import sent_tokenize
from huggingface_hub import login
from dotenv import load_dotenv
from rouge import Rouge
from tqdm import tqdm


In [ ]:
cleaned_fn = "cleaned.json"
ocred_fn = "original_ocr.json"

load_dotenv()
directory_path = os.environ['DATA_PATH']
login(token=os.environ['HF_TOKEN'])
genai.configure(api_key=os.environ['GEMINI_KEY']) 


In [ ]:
datasetRaw = {}

for name in [ocred_fn, cleaned_fn]:
    if name not in os.listdir(directory_path):
        print(f"File [{name}] not Found in directory [{directory_path}]")

    file_path = os.path.join(directory_path, name)
    with open(file_path, 'r') as file:
        datasetRaw[i.split('.')[0]] = json.load(file)
        file.close()

## Minerva 1B

In [ ]:
model_name = "sapienzanlp/Minerva-1B-base-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, device_map='cuda', torch_dtype=torch.float16
)